In [155]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn import svm
from Utils import GeoUtils

from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA

import time

%matplotlib inline

In [156]:
def getDistance(real, predict):
    lat1 = real[:,0]
    lon1 = real[:,1]
    lat2 = predict[:,0]
    lon2 = predict[:,1]

    return np.array(GeoUtils.GeoUtils.distanceInKm(lat1, lon1,lat2, lon2))

In [157]:
data = pd.read_csv("Dados/LocTreino_Equipe_4.csv")

bts_data = pd.read_csv("Dados/dados_BTSs.csv")

data.drop_duplicates(inplace=True)
data.shape
#data = data.drop(['pontoId'], axis=1)
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())
data['taBTS4'] = data['taBTS4'].fillna(data['taBTS4'].mean())
data['pathBTS1'] = data['pathBTS1'].fillna(data['pathBTS1'].mean())

data.describe()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,1471.292667,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,849.057388,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,2.000000,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,746.750000,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,1464.500000,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,2190.250000,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,2955.000000,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [158]:
x = data.drop(['lat', 'lon', 'pontoId'], axis=1)

transformerX = StandardScaler().fit(x)
x_scaled = transformerX.transform(x)
x.describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [159]:
y = pd.DataFrame(columns=['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6'])

bts1_coord = np.tile(bts_data[['lat', 'lon']].iloc[0].values, (1500,1))
y['distBTS1'] = getDistance(data[['lat', 'lon']].values, bts1_coord) 

bts2_coord = np.tile(bts_data[['lat', 'lon']].iloc[1].values, (1500,1))
y['distBTS2'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts3_coord = np.tile(bts_data[['lat', 'lon']].iloc[2].values, (1500,1))
y['distBTS3'] = getDistance(data[['lat', 'lon']].values, bts3_coord) 

bts4_coord = np.tile(bts_data[['lat', 'lon']].iloc[3].values, (1500,1))
y['distBTS4'] = getDistance(data[['lat', 'lon']].values, bts4_coord) 

bts5_coord = np.tile(bts_data[['lat', 'lon']].iloc[4].values, (1500,1))
y['distBTS5'] = getDistance(data[['lat', 'lon']].values, bts5_coord) 

bts6_coord = np.tile(bts_data[['lat', 'lon']].iloc[5].values, (1500,1))
y['distBTS6'] = getDistance(data[['lat', 'lon']].values, bts6_coord) 

y.head()

,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
0,0.879576,0.191390,1.294424,0.191390,1.291290,1.219177
1,0.665594,0.395403,1.309441,0.395403,1.103513,0.952860
2,0.629140,0.408747,1.339021,0.408747,1.067447,0.919427
3,1.026196,0.716060,0.883794,0.716060,1.471446,1.172785
4,0.793764,1.621548,2.642108,1.621548,0.390252,0.716741


In [160]:
transformerY = StandardScaler().fit(y)
y_scaled = transformerY.transform(y)

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.01)

X_test = x_scaled
y_test = y_scaled

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

invX = transformerX.inverse_transform
invY = transformerY.inverse_transform

(1485, 12) (1500, 12) (1485, 6) (1500, 6)


In [165]:
knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn.fit(X_train, y_train)

knnreal = invY(y_test)
knnpredict = invY(knn.predict(X_test))

print (np.mean((knnpredict - knnreal)**2, axis=0))

print (np.std((knnpredict - knnreal)**2, axis=0))

[0.00074125 0.00183044 0.00091165 0.00183044 0.00078657 0.00075033]
[0.00698648 0.04365078 0.00775888 0.04365078 0.00504051 0.00301865]


In [166]:
bts1_coord = np.tile(bts_data[['lat', 'lon']].iloc[0].values, (1500,1))
data['distBTS1'] = getDistance(data[['lat', 'lon']].values, bts1_coord) 

bts2_coord = np.tile(bts_data[['lat', 'lon']].iloc[1].values, (1500,1))
data['distBTS2'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts3_coord = np.tile(bts_data[['lat', 'lon']].iloc[2].values, (1500,1))
data['distBTS3'] = getDistance(data[['lat', 'lon']].values, bts3_coord) 

bts4_coord = np.tile(bts_data[['lat', 'lon']].iloc[3].values, (1500,1))
data['distBTS4'] = getDistance(data[['lat', 'lon']].values, bts4_coord) 

bts5_coord = np.tile(bts_data[['lat', 'lon']].iloc[4].values, (1500,1))
data['distBTS5'] = getDistance(data[['lat', 'lon']].values, bts5_coord) 

bts6_coord = np.tile(bts_data[['lat', 'lon']].iloc[5].values, (1500,1))
data['distBTS6'] = getDistance(data[['lat', 'lon']].values, bts6_coord) 

x2 = data.drop(['lat', 'lon'], axis=1)
x2.describe()

,pontoId,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,1471.292667,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560,0.651671,0.824295,1.673787,0.824295,0.894737,0.836718
std,849.057388,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692,0.338396,0.429586,0.489971,0.429586,0.451967,0.348651
min,2.000000,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.054158,0.009910,0.870120,0.009910,0.026198,0.031430
25%,746.750000,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000,0.372980,0.527395,1.314656,0.527395,0.476412,0.591253
50%,1464.500000,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000,0.603009,0.760281,1.669549,0.760281,0.858269,0.811690
75%,2190.250000,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000,0.901601,1.023290,1.989360,1.023290,1.289275,1.097895
max,2955.000000,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000,1.847645,2.274950,2.835246,2.274950,2.053067,1.527323


In [167]:
y2 = data[['lat', 'lon']]
y2.describe()

,lat,lon
count,1500.000000,1500.000000
mean,-8.070097,-34.894814
std,0.004232,0.004354
min,-8.077570,-34.906830
25%,-8.073197,-34.897387
50%,-8.069940,-34.895346
75%,-8.067113,-34.891169
max,-8.059339,-34.885067


In [168]:
transformerY2 = StandardScaler().fit(y2)
y2_scaled = transformerY2.transform(y2)

X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size = 0.01)
print(X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape)

invX2 = transformerX2.inverse_transform
invY2 = transformerY2.inverse_transform
X_test2.head()

(1485, 19) (15, 19) (1485, 2) (15, 2)


NameError: name 'transformerX2' is not defined

In [152]:
x_aux = pd.DataFrame()
x_aux = X_test2.drop(['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6', 'pontoId'], axis=1)
predicts = invY(knn.predict(transformerX.transform(x_aux.values)))
X_test2[['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6']] = predicts
X_test2.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,pontoId,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
10,1339,98.69,120.723333,121.89,105.656667,104.99,108.99,0,1.0,3,1,1,1,0.244579,0.619667,1.761269,0.619667,0.645165,0.675653
124,2830,121.59,129.623333,139.49,128.190000,136.49,142.09,1,2.0,4,2,0,1,0.756156,1.583635,2.607991,1.583635,0.353172,0.690022


In [64]:
trans_X = StandardScaler().fit(x2.drop(['pontoId'], axis=1))
X_train2_scaled = trans_X.transform(X_train2.drop(['pontoId'], axis=1))
X_test2_scaled = trans_X.transform(X_test2.drop(['pontoId'], axis=1))

trans_Y = StandardScaler().fit(y2)

y_train2 = trans_Y.transform(y_train2)
y_test2 = trans_Y.transform(y_test2)

invY2 = trans_Y.inverse_transform
invX2 = trans_X.inverse_transform

knn2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn2.fit(X_train2_scaled, y_train2)

knn2real = invY2(y_test2)
start = time.time()
knn2predict = invY2(knn2.predict(X_test2_scaled))
end = time.time()
print(end - start)
print(getDistance(knn2real, knn2predict).mean())

0.0019109249114990234
0.02818740426546629


In [ ]:
print(getDistance(knn2real, knn2predict).std())
print(getDistance(knn2real, knn2predict).min())
print(getDistance(knn2real, knn2predict).max())

In [ ]:
plt.title('Histogram of errors')
plt.ylabel('# of samples')
plt.xlabel('Error (m)')

print(kridgepredict.shape)

plt.hist(getDistance(knn2real, knn2predict)*1000, 30)

In [ ]:
gboost = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=50, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5))
gboost.fit(X_train2, y_train2)

gboostreal = invY2(y_test2)
gboostpredict = invY2(gboost.predict(X_test2))

print(getDistance(gboostreal, gboostpredict).mean())

In [ ]:
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train2,y_train2)

svmreal = invY2(y_test2)
svmpredict = invY2(regr.predict(X_test2))

print(getDistance(svmreal, svmpredict).mean())

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
multioutputregressor = MultiOutputRegressor(model_xgb).fit(X_train2, y_train2)

realoutp = invY2(multioutputregressor.predict(X_test2))
realoutr = invY2(y_test2)                                         

print(getDistance(realoutr, realoutp).mean())

In [154]:
import time
kridge = MultiOutputRegressor(KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5))
kridge.fit(X_train2_scaled, y_train2)
kridgereal = invY2(y_test2)
start = time.time()
kridgepredict = invY2(kridge.predict(X_test2_scaled))
end = time.time()
print(end - start)
print(getDistance(kridgereal, kridgepredict).mean())

NotFittedError: This MultiOutputRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [66]:
data_final = pd.read_csv("Dados/LocTest.csv")
data_final['taBTS2'] = data_final['taBTS2'].fillna(data_final['taBTS2'].mean())
data_final['pathBTS1'] = data_final['pathBTS1'].fillna(data_final['pathBTS1'].mean())
data_final.describe()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,1429.730000,-8.070378,-34.894895,120.476043,119.254919,124.059041,119.573235,120.931101,130.893249,0.645000,1.094973,2.485000,0.955000,1.130000,1.105000
std,812.821846,0.004010,0.004296,46.520346,11.771553,8.559135,39.410691,12.480747,52.726890,0.671737,1.393259,0.961456,0.738717,0.840615,0.645023
min,8.000000,-8.077541,-34.905296,87.723333,79.423333,92.690000,87.856667,87.023333,87.523333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,685.000000,-8.073634,-34.898285,108.052500,112.481667,119.081667,107.765000,112.215000,113.319167,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000
50%,1503.000000,-8.069830,-34.895534,117.456667,119.456667,124.115000,116.390000,122.390000,129.406667,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
75%,2008.500000,-8.068513,-34.891111,124.615000,128.577500,129.783750,124.365000,130.231667,136.365000,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000
max,2928.000000,-8.060637,-34.885357,653.594803,143.390000,143.490000,604.404762,144.490000,625.087259,2.000000,11.864499,5.000000,3.000000,3.000000,2.000000


In [70]:
x_final = data_final.drop(['lat', 'lon', 'pontoId'], axis=1)

predicts = invY(knn.predict(transformerX.transform(x_aux.values)))

x_final.reindex()

x_final[['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6']] = predicts

x_final.head()

TypeError: only integer scalar arrays can be converted to a scalar index

In [47]:
x_final.describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,120.476043,119.254919,124.059041,119.573235,120.931101,130.893249,0.645000,1.094973,2.485000,0.955000,1.130000,1.105000,0.661707,0.819368,1.663656,0.819368,0.902486,0.858238
std,46.520346,11.771553,8.559135,39.410691,12.480747,52.726890,0.671737,1.393259,0.961456,0.738717,0.840615,0.645023,0.328079,0.398738,0.506834,0.398738,0.452574,0.328329
min,87.723333,79.423333,92.690000,87.856667,87.023333,87.523333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.080453,0.078577,0.872378,0.078577,0.069393,0.091524
25%,108.052500,112.481667,119.081667,107.765000,112.215000,113.319167,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000,0.376636,0.563850,1.308791,0.563850,0.453549,0.662950
50%,117.456667,119.456667,124.115000,116.390000,122.390000,129.406667,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.600206,0.772423,1.620809,0.772423,0.905332,0.798398
75%,124.615000,128.577500,129.783750,124.365000,130.231667,136.365000,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000,0.886250,1.002688,2.000042,1.002688,1.289583,1.182646
max,653.594803,143.390000,143.490000,604.404762,144.490000,625.087259,2.000000,11.864499,5.000000,3.000000,3.000000,2.000000,1.582818,2.015779,2.798273,2.015779,1.858166,1.523075


In [52]:
y_final = data_final[['lat', 'lon']]

# trans_X = StandardScaler().fit(x_final)
# trans_Y = StandardScaler().fit(y_final)

# x_final_scaled = StandardScaler().fit(x_final).transform()

x_final_scaled = trans_X.transform(x_final)
y_final_scaled = trans_Y.transform(y_final)

# invY2 = trans_Y.inverse_transform
# invX2 = trans_X.inverse_transform

# print(x_final_scaled)

kridgereal = invY2(y_final_scaled)
kridgepredict = invY2(kridge.predict(x_final_scaled))
print(y_final)
print(kridgereal)
print(kridgepredict)

print(getDistance(kridgereal, kridgepredict).mean())

          lat        lon
0   -8.069616 -34.894836
1   -8.069243 -34.888069
2   -8.069219 -34.900562
3   -8.068377 -34.890717
4   -8.068587 -34.897018
5   -8.065628 -34.892662
6   -8.068026 -34.890564
7   -8.076082 -34.899818
8   -8.069702 -34.898060
9   -8.073581 -34.897671
10  -8.070419 -34.893299
11  -8.068716 -34.897636
12  -8.075210 -34.892113
13  -8.068727 -34.889603
14  -8.071367 -34.891891
15  -8.072575 -34.896355
16  -8.069393 -34.889519
17  -8.075513 -34.893074
18  -8.063748 -34.893967
19  -8.068761 -34.889000
20  -8.070368 -34.896942
21  -8.076043 -34.899834
22  -8.074183 -34.890305
23  -8.063772 -34.897038
24  -8.072496 -34.896748
25  -8.073058 -34.896587
26  -8.068021 -34.890598
27  -8.071483 -34.900780
28  -8.077169 -34.898254
29  -8.069755 -34.897926
..        ...        ...
170 -8.065022 -34.885532
171 -8.077349 -34.898785
172 -8.062394 -34.887768
173 -8.076345 -34.899746
174 -8.070271 -34.893581
175 -8.068880 -34.900177
176 -8.077239 -34.899406
177 -8.065791 -34.890293


In [53]:
# trans_X = StandardScaler().fit(x2.drop(['pontoId'], axis=1))
# X_train2_scaled = trans_X.transform(x_final.drop(['pontoId'], axis=1))
# X_test2_scaled = trans_X.transform(x_final.drop(['pontoId'], axis=1))

# trans_Y = StandardScaler().fit(y2)

# y_train2 = trans_Y.transform(y_train2)
# y_test2 = trans_Y.transform(y_test2)

# invY2 = trans_Y.inverse_transform
# invX2 = trans_X.inverse_transform

# knn2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
# knn2.fit(X_train2_scaled, y_train2)

knn2real = invY2(y_final_scaled)
start = time.time()
knn2predict = invY2(knn2.predict(x_final_scaled))
end = time.time()
print(end - start)
print(getDistance(knn2real, knn2predict).mean())

0.014494180679321289
0.10961817594345485


In [ ]:
print(getDistance(kridgereal, kridgepredict).std())
print(getDistance(kridgereal, kridgepredict).min())
print(getDistance(kridgereal, kridgepredict).max())

In [ ]:
plt.title('Histogram of errors')
plt.ylabel('# of samples')
plt.xlabel('Error (m)')

print(kridgepredict.shape)

plt.hist(getDistance(kridgereal, kridgepredict)*1000, 30)

In [ ]:
knnonly_errors = np.load('knn_distances.npy')
print(knnonly_errors.shape)
print(knnknn_errors.shape)
print(knnkrdg_errors.shape)
knnknn_errors = getDistance(knn2real, knn2predict)*1000
knnkrdg_errors = getDistance(kridgereal, kridgepredict)*1000

allerrors = []

allerrors.extend(knnonly_errors)
allerrors.extend(knnknn_errors)
allerrors.extend(knnkrdg_errors)

labels = []
labels.extend(np.repeat('KNN Direto',150))
labels.extend(np.repeat('KNN -> KNN',150))
labels.extend(np.repeat('KNN -> Kridge',150))

df_data = {'Error (m)': allerrors,
           'Approach': labels}
df = pd.DataFrame(df_data)
plt.title('Boxplots')
sns.boxplot(x='Approach', y='Error (m)', data=df,width=0.8)
plt.show()

In [ ]:
import scipy.stats as scp


scp.stats.friedmanchisquare(knnonly_errors,
                            knnknn_errors,
                            knnkrdg_errors)

In [ ]:
import folium 

maap = folium.Map(location=[y2['lat'].mean(),
                                    y2['lon'].mean()],
                          zoom_start=13,
                          tiles='CartoDB dark_matter')

for i, (pos, pred_pos) in enumerate(zip(kridgereal,
                                        kridgepredict)):
    marker = folium.CircleMarker(location=pos,
                                 color='blue',
                                 weight=2,
                                 radius=1,
                                 fill_color='blue',
                                 fill=True)
    marker.add_to(maap)
    marker = folium.CircleMarker(location=pred_pos,
                                 color='yellow',
                                 weight=2,
                                 radius=1,
                                 fill_color='yellow',
                                 fill=True)
    marker.add_to(maap)
    
    folium.PolyLine([pos, pred_pos],
                    color="white",
                    weight=1,
                    opacity=0.5).add_to(maap)
    
maap

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=3)
knn2 = KNeighborsRegressor(n_neighbors=3)

gboost = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=50, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5)
svr = svm.SVR()
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
gboost2 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5)

gboost3 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=5, min_samples_split=10, 
                                   loss='huber', random_state =5)

lasso = Lasso(alpha =0.0005, random_state=1)

stack = StackingAveragedModels(base_models = (gboost, svr, knn), meta_model=lasso)

stack2 = MultiOutputRegressor(StackingAveragedModels(base_models = (stack, knn2, gboost2), meta_model=gboost3))

stack2.fit(X_train2, y_train2)

stackreal = invY2(y_test2)
stackpredict = invY2(stack2.predict(X_test2))

print(getDistance(stackreal, stackpredict).mean())

In [ ]:
print(X_test2['pontoId'].shape)
print(kridgereal[0].shape)
print(kridgereal[1].shape)
print(kridgepredict)

result_dict = {'pontoId' : X_test2['pontoId'].values,
            'lat' :  kridgereal[0],
            'lon' : kridgereal[1],
            'lat_pred' : kridgepredict[:,0],
            'lon_pred' : kridgepredict[:,1],
            'erro_loc' : getDistance(kridgereal, kridgepredict)}

resultDF = pd.DataFrame(result_dict)
resultDF.to_csv('Resultados.csv')